In [1]:
import sys
from pathlib import Path
from os import getcwd

from datetime import date, timedelta

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input


---------------------------------------------------------------

root: /home/vinnylg/projects/covid19datascience/bulletin
parent: /home/vinnylg/projects/covid19datascience
default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output

---------------------------------------------------------------



In [4]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
# municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [5]:
# tb_pacientes = TbPacientes()
# tb_pacientes.update()

casost = pd.read_pickle('tb_pacientes.pkl')#tb_pacientes.get()
casost['nome'] = casost['nome'].apply(lambda nome: ''.join(filter(lambda x: ('A' <= x <= 'Z') or ( x == ' '), nome)))
casost = casost.sort_values(['dt_notificacao','nome','idade'])
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
# casost = casost.loc[casost['dt_notificacao'] <= '2021-05-27'].copy()
print(f"casost len: {len(casost)}\n")
casost.groupby('obito')[['sexo']].count()

casost len: 1084380

casost len: 1084380



,sexo
obito,
NAO,1058212
SIM,26168


In [6]:
casost

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito
8841,8838,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-04-03 00:00:00,Notificações,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,47,NaT,NaT,48,46,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,NaN
10906,10932,410690,410690,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,43,NaT,NaT,44,42,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU12032020,NaN
10907,10933,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,15,NaT,NaT,16,14,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR12032020,NaN
11440,11470,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,NaN,07/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,26,NaT,NaT,27,25,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE16032020,NaN
11462,11492,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,NaN,08/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,26,NaT,NaT,27,25,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN18032020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587803,1176616,411820,999999,ZENILTON PIRES MODESTO,M,39,PARANAGUA,IGNORADO/EXTERIOR/IG,"COVID-19, Biologia Molecular",2021-05-17,2021-05-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130.0,39,NaT,NaT,40,38,ZENILTONPIRESMODESTO39PARANAGUA,ZENILTONPIRESMODESTO38PARANAGUA,ZENILTONPIRESMODESTO40PARANAGUA,ZENILTONPIRESMODESTO39IGNORADOEXTERIORIG,ZENILTONPIRESMODESTO38IGNORADOEXTERIORIG,ZENILTONPIRESMODESTO40IGNORADOEXTERIORIG,ZENILTONPIRESMODESTO17052021,NaN
587804,1176617,410720,410720,ZEUNILDE AGUIAR ACERM

In [7]:
casost.groupby('status')[['sexo']].count()

,sexo
status,
Recuperado,755494
recuperado,2


In [8]:
casosct = casost.copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

In [9]:
casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_atualizacao']

In [10]:
casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'dt_obito']
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_com_recuperado'] 

In [11]:
casosct.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1,755496
2,26168
3,302716


In [12]:
casosct['uf_resid'] = 'PR'
casosct['ibge_res_pr'] = casosct['ibge_res_pr'].apply(int)
casosct.loc[casosct['ibge_res_pr']==999999,'uf_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[-1])
casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[0])

casos_fora = pd.merge(casosct.loc[casosct['ibge_res_pr']==999999].drop(columns=['ibge_res_pr']),municipios[['ibge','mun_resid','uf']].rename(columns={'uf':'uf_resid','ibge':'ibge_res_pr'}), on=['mun_resid','uf_resid'], how='left')
# casos_fora.loc[casos_fora['ibge_res_pr'].isna(),'excluir'] = 'SIM'
casos_fora.loc[casos_fora['ibge_res_pr'].isna(),'ibge_res_pr'] = 999999
casos_fora = casos_fora.set_index('identificacao')
casosct = casosct.set_index('identificacao')
casosct.update(casos_fora)

casosct = casosct.reset_index()

casosct['ibge_res_pr'] = casosct['ibge_res_pr'].apply(int)
casosct['idade'] = casosct['idade'].astype(int)
casosct

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito,data_evolucao,data_com_evolucao,uf_resid
0,8838,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-04-03 00:00:00,Notificações,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,23.0,47.0,NaT,NaT,48.0,46.0,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,NaN,2020-04-03 00:00:00,NaT,PR
1,10932,410690,410690,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,23.0,43.0,NaT,NaT,44.0,42.0,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU12032020,NaN,NaN,NaT,PR
2,10933,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,23.0,15.0,NaT,NaT,16.0,14.0,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR12032020,NaN,NaN,NaT,PR
3,11470,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,NaN,07/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,14.0,26.0,NaT,NaT,27.0,25.0,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE16032020,NaN,NaN,NaT,PR
4,11492,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,NaN,08/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,14.0,26.0,NaT,NaT,27.0,25.0,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN18032020,NaN,NaN,NaT,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084375,1176616,411820,999999,ZENILTON PIRES MODESTO,M,39,PARANAGUA,IGNORADO/EXTERIOR/IG,"COVID-19, Biologia Molecular",2021-05-17,2021-05-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,130.0,39.0,NaT,NaT,40.0,38.0,ZENILTONPIRESMODESTO39PARANAGUA,ZENILTONPIRESMODESTO38PARANAGUA,ZENILTONPIRESMODESTO40PARANAGUA,ZENILTONPIRESMODESTO39IGNORADOEXTERIORIG,ZENILTONPIRESMODESTO38I

In [13]:
casosct.groupby(['evolucao'])[['sexo']].count()

,sexo
evolucao,
1.0,755496
2.0,26168
3.0,302716


In [14]:
casosct['id_notifica'] = -1
casosct['ibge_resid'] = casosct['ibge_res_pr']
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(int)
casosct['comunicacao'] = casosct['dt_notificacao']
casosct['is'] = casosct['dt_inicio_sintomas']

casosct = casosct[['identificacao', 'id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao', 'data_com_evolucao']]
casosct

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaN,1.0,2020-04-03 00:00:00,NaT
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,1.0,NaN,NaT
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,1.0,NaN,NaT
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,1.0,NaN,NaT
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,1.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084375,1176616,-1,PR,411820,999999,ZENILTON PIRES MODESTO,M,39,"COVID-19, Biologia Molecular",2021-05-17,2021-05-28,NaN,3.0,NaT,NaT
1084376,1176617,-1,PR,410720,410720,ZEUNILDE AGUIAR ACERMAO,F,37,"COVID-19, Biologia Molecular",2021-05-27,2021-05-28,2021-05-17 00:00:00,3.0,NaT,NaT
1084377,1176618,-1,PR,411840,411840,ZILDA LOPES,F,48,"COVID-19, Biologia Molecular",2021-05-16,2021-05-28,2021-05-09 00:00:00,3.0,NaT,NaT
1084378,1176619,-1,PR,411840,411840,ZILDA MACHADO PAZZETTO DOS SANTOS,F,42,Teste Rápido Antígeno (Imunocromatografia),2021-05-27,2021-05-28,2021-05-24 00:00:00,3.0,NaT,NaT


In [15]:
cc = CasosConfirmados()
cc.df = casosct
cc.save('cc_28_05_2021')

databases: ['cc_21_05_2021', 'cc_22_05_2021', 'cc_23_05_2021', 'cc_25_05_2021', 'cc_27_05_2021']
Load /home/vinnylg/projects/covid19datascience/bulletin/database/casos_confirmados/cc_28_05_2021.pkl
saving Casos Confirmados to pkl time elepsed:00:00:00


In [16]:
casosct.loc[casosct['evolucao']==2]

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao
104,4062,-1,PR,411520,411520,BELCHIOR MARTINS,M,84,LACEN,2020-03-27,2020-03-27,2020-03-15 00:00:00,2.0,2020-03-26 00:00:00,2020-03-26
105,6249,-1,PR,410690,410690,CLODOALDO PEDRO BIASOTTO,M,72,LACEN,2020-03-27,2020-03-27,NaN,2.0,2020-04-07 00:00:00,2020-04-07
108,10853,-1,PR,410940,410940,FERNANDO JOSE COSTA,M,57,LACEN,2020-03-26,2020-03-27,2020-03-08 00:00:00,2.0,2020-06-15 00:00:00,2020-06-15
111,11830,-1,PR,411080,411080,GERSON LACERDA DE SOUZA,M,51,LACEN,2020-03-27,2020-03-27,NaN,2.0,2020-04-10 00:00:00,2020-04-10
119,27696,-1,PR,411520,412625,ROSANGELA ANTUNES MACHADO DA,F,54,LACEN,2020-03-27,2020-03-27,2020-03-14 00:00:00,2.0,2020-03-25 00:00:00,2020-03-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083727,1175968,-1,PR,410480,410480,SANDRA DIAS PINTO MACHADO,F,50,Vírus Respiratório,2021-05-26,2021-05-28,2021-05-15 00:00:00,2.0,2021-05-23 00:00:00,2021-05-28
1083770,1176011,-1,PR,412050,412050,SEBASTIAO BENEDITO FERREIRA,M,73,"COVID-19, Biologia Molecular",2021-05-21,2021-05-28,2021-05-11 00:00:00,2.0,2021-05-26 00:00:00,2021-05-28
1083827,1176068,-1,PR,412750,412750,SILMARA DO ROCIO EUZEBIO,F,51,"COVID-19, Biologia Molecular",2021-03-06,2021-05-28,2021-02-28 00:00:00,2.0,2021-03-15 00:00:00,2021-05-28
1084023,1176264,-1,PR,411990,411990,TEREZINHA ALVES DA CRUZ,F,68,"COVID-19, Biologia Molecular",2021-04-12,2021-05-28,2021-03-28 00:00:00,2.0,2021-05-21 00:00:00,2021-05-28
